In [10]:
import folium
import requests
import json
import polyline
import pandas as pd
from geopy.geocoders import Nominatim

In [5]:
df = pd.read_csv('postcodes_test.csv')

In [7]:
df.head(20)

,team,ground,postcode,county_fa,latitude,longitude
0,AFC Bournemouth,Vitality Stadium,BH7 7AF,Hampshire,50.735313,-1.839234
1,Arsenal,Emirates Stadium,N5 1BU,London,51.556667,-0.106371
2,Aston Villa,Villa Park,B6 6HE,Birmingham,52.508486,-1.884946
3,Brentford,Brentford Community Stadium,TW8 9BW,London,51.491117,-0.290548
4,Brighton & Hove Albion,Amex Stadium,BN1 9BL,Sussex,50.860916,-0.084280
5,Chelsea,Stamford Bridge,SW6 1HS,Surrey,51.481124,-0.190297
6,Crystal Palace,Selhurst Park,SE25 6PU,Surrey,51.397894,-0.085923
7,Everton,Goodison Park,L4 4EL,Lancashire,53.439197,-2.967017
8,Fulham,Craven Cottage,SW6 6HH,London,51.474520,-0.220610
9,Leeds United,Elland Road,LS11 0ES,West Riding,53.777714,-1.573104


In [29]:
# Define the team name you want to match
central_team = 'Nottingham Forest'

# Iterate over the dataframe rows
for index, row in df.iterrows():
    # Check if the team name matches the target team
    if row["team"] == central_team:
        # Get the postcode for the matching team
        central_location = row["postcode"]
        # Print or store the postcode as needed
        print("Postcode for", central_team, ":", central_location)

Postcode for Nottingham Forest : NG2 5FJ


In [22]:
geolocator = Nominatim(user_agent="postcode_geocoder")
central_coords = geolocator.geocode(central_location)
central_lat, central_lon = central_coords.latitude, central_coords.longitude


In [23]:
def geocode_postcode(postcode):
    location = geolocator.geocode(postcode)
    return location.latitude, location.longitude

df["Coordinates"] = df["postcode"].apply(geocode_postcode)

In [46]:
import folium
import requests
import json
import polyline

def get_distance(point1, point2, map):
    """
    Gets the driving distance and duration between two points using OSRM.

    Arguments:
    - point1 (dict): A dictionary representing the latitude and longitude of the first point.
    - point2 (dict): A dictionary representing the latitude and longitude of the second point.
    - map (folium.Map): The map object to display the driving route.

    Returns:
    - tuple: A tuple containing the distance (in meters) and duration (in seconds) of the route.
    """
    url = (
        "http://router.project-osrm.org/route/v1/driving/"
        f"{point1['longitude']},{point1['latitude']};"
        f"{point2['longitude']},{point2['latitude']}"
        "?overview=full&alternatives=false"
    )
    r = requests.get(url)

    # Get the route geometry from the returned values
    route_geometry = json.loads(r.content)["routes"][0]["geometry"]

    # Decode the route geometry using the polyline library
    decoded_route = polyline.decode(route_geometry)

    # Add the driving route to the map
    folium.PolyLine(
        locations=decoded_route,
        weight=2.5,
        opacity=1.0,
    ).add_to(map)

    # Get the distance and duration from the returned values
    route_distance = json.loads(r.content)["routes"][0]["distance"]
    route_duration = json.loads(r.content)["routes"][0]["duration"]

    return route_distance, route_duration


# Create a map centered on the central location
map = folium.Map(location=[central_lat, central_lon], zoom_start=10)

total_distance = 0

for index, row in df.iterrows():
    central_coords = {"latitude": central_lat, "longitude": central_lon}
    postcode_coords = {"latitude": row["Coordinates"][0], "longitude": row["Coordinates"][1]}
    distance, _ = get_distance(central_coords, postcode_coords, map)
    distance = round(distance * 0.000621371, 1)

    total_distance += distance

    # Create the label to display postcode, team, and driving distance information
    label = f"postcode: {row['postcode']}<br>Team: {row['team']}<br>Distance: {distance} miles"
    
    # Create the tooltip content for the central location marker
    tooltip_content = f"Team: {central_team}<br>Postcode: {central_location}<br>Total Distance: {total_distance} miles"

    # Add the marker with the label and team name displayed above
    # Add the marker with the label displayed on hover
    folium.Marker(
        location=row["Coordinates"],
        tooltip=label,
        icon=folium.Icon(color="blue", icon="info-sign"),
    ).add_to(map)

    folium.Marker(
        location=[central_lat, central_lon],
        tooltip=tooltip_content,
        icon=folium.Icon(color="red", icon="info-sign"),
    ).add_to(map)



# Save the map to an HTML file
map.save("postcodes_map.html")

print(total_distance)

2219.8
